<a href="https://colab.research.google.com/github/Pablo2710/ProyectoDSI/blob/main/ProyectoDSParteI_Martinez.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Financial Fraud Detection**

Este Dataset contiene 5 millones de transacciones financieras generadas  mediante un script de Python. La finalidad es simular el comportamiento real en la investigación de detección de fraudes y aplicaciones de aprendizaje automático. Cada registro de transacción incluye campos como:

**Detalles de la transacción:** ID, marca de tiempo, cuentas del remitente/receptor, monto, tipo (depósito, transferencia, etc.)

**Características de comportamiento:** tiempo transcurrido desde la última transacción, puntuación de desviación del gasto, puntuación de velocidad, puntuación de anomalías geográficas

**Metadatos:** ubicación, dispositivo utilizado, canal de pago, dirección IP, hash del dispositivo

**Indicadores de fraude:** etiqueta binaria de fraude (is_fraud) y tipo de fraude (por ejemplo, lavado de dinero, apropiación de cuentas)

El conjunto de datos sigue patrones de fraude realistas y anomalías de comportamiento, lo que lo hace adecuado para:

- *Modelos de clasificación binaria y multiclase*

- *Sistemas de detección de fraude*

- *Detección de anomalías en series temporales*

- *Ingeniería de características y explicabilidad del modelo*